# Calculating momentum based on 12-month lookback period
> To calculate the cumulative return over the past 12 months, we take the net return streams from each month and turn them into gross returns by adding 1. Thus, if Apple's net returns for January are –10.77 percent, Apple's gross returns for January are 0.8923 (–0.1077 + 1). Then, we multiply all the gross return series (i.e., months) and subtract 1 to find the cumulative 12-month net return. ([Location 2467](https://readwise.io/to_kindle?action=open&asin=B01LY6P2LB&location=2467))

In [1]:
%load_ext dotenv
%dotenv

In [8]:
import arrow
import pandas as pd
import numpy as np

import dataclasses
import math
import sys
sys.path.append('../src')

In [9]:
import quality_momentum as qm

In [10]:
import importlib; importlib.reload(qm)

<module 'quality_momentum' from '/quality-momentum/notebooks/../src/quality_momentum/__init__.py'>

In [14]:
now = arrow.get('2017-12-03')
equities = qm.calculate_momentum.get_universe_of_equities(now)
momentum_measures = [qm.calculate_momentum.get_monthly_momentum(e, now) for e in equities]
df = pd.DataFrame.from_records([dataclasses.asdict(x) for x in momentum_measures], index='ticker')
df['quantile_rank'] = pd.qcut(df['momentum'], 10, labels=False)

In [15]:
top_decile_momentum_equities = df[df['quantile_rank'] == 9]
top_quality_momentum = pd.qcut(top_decile_momentum_equities['fip'], 2, labels=['high_quality', 'low_quality'])
top_decile_momentum_equities = top_decile_momentum_equities.assign(quality_momentum=top_quality_momentum.values)
# equities_to_buy = top_decile_momentum_equities[top_decile_momentum_equities['quality_momentum'] == 1]
equities_to_buy = top_decile_momentum_equities[top_decile_momentum_equities['quality_momentum'] == 'high_quality']

In [16]:
print(equities_to_buy)

        momentum       fip  quantile_rank quality_momentum
ticker                                                    
BA      0.734723 -0.185345              9     high_quality


In [18]:
now = arrow.get('2017-12-03')
tickers = qm.calculate_momentum.get_quality_momentum_stocks(now, 2)

In [19]:
tickers = equities_to_buy
available_cash = 100000

In [20]:
data = {x: available_cash / len(tickers) for x in tickers}
df = pd.DataFrame.from_dict(data, orient="index")

In [21]:
df.index

Index(['momentum', 'fip', 'quantile_rank', 'quality_momentum'], dtype='object')

In [22]:
df.head(10)

,0
momentum,100000.0
fip,100000.0
quantile_rank,100000.0
quality_momentum,100000.0


In [23]:
portfolio = qm.portfolio.purchase_new_shares(arrow.get('2017-12-03'), 100000, 4, qm.portfolio.WeightType.equal_weighted)

In [24]:
print(portfolio.head())

      position_size
BA            25000
FB            25000
AAPL          25000
MCD           25000


In [25]:
now = arrow.get('2017-12-03')
import pandas_datareader as pdr

equities = {}
for equity in portfolio.index.tolist():
    # example from https://nbviewer.ipython.org/github/twiecki/financial-analysis-python-tutorial/blob/master/1.%20Pandas%20Basics.ipynb
    adj_close_series = pdr.quandl.QuandlReader(equity, now.shift(days=-2).format('YYYY-MM-DD'), now.format('YYYY-MM-DD')).read()['AdjClose']
    # grab first adjusted close value available
    equities[equity] = {"adj_close": adj_close_series[0]}
df = pd.DataFrame.from_dict(data=equities, orient='index')

In [26]:
df

,adj_close
AAPL,171.05
BA,271.38
FB,175.10
MCD,172.87


In [27]:
portfolio = portfolio.join(df)

In [30]:
portfolio

,position_size,adj_close
BA,25000,271.38
FB,25000,175.10
AAPL,25000,171.05
MCD,25000,172.87


In [31]:
portfolio['num_shares_to_purchase'] = (portfolio['position_size'] / portfolio['adj_close']).apply(np.floor)

In [32]:
portfolio

,position_size,adj_close,num_shares_to_purchase
BA,25000,271.38,92.0
FB,25000,175.10,142.0
AAPL,25000,171.05,146.0
MCD,25000,172.87,144.0
